## Tutorial
This tutorial aims to present how to obtain vector representation of a graph using anonymous walks. In this tutorial we will show: 
* How to obtain anonymous walk distribution of a graph. 
* How to compute graph kernel from embeddings of graphs. 
* How to evaluate performance of kernel matrix in graph classification task. 

Dependencies: 
* networkx (v. 1.11+)
* tensorflow (v. 1.4.0+)
* scikit-learn (v. 0.18.1+)

### Reading graphs

`read_graphs` function allows to read graphs as networkx Digraph object. The input can be any of the 2 options: 
1. Folder, where graphs are located and their file extensions.
2. List of all filenames. 

In the first case, graph filenames should have an index (e.g. graph1.graphml or mutag_20.graphml). This index will be automatically used by the classifier to link it with graph class. 

In [2]:
from AnonymousWalkKernel import GraphKernel

gk = GraphKernel()
gk.read_graphs(folder = '../bio/mutag/', ext = 'graphml')

print('Number of graphs: {}'.format(len(gk.graphs)))

Number of graphs: 188


### Getting feature-based embedding of a graph
`AnonymousWalks` class contains functions to calculate distribution of anonymous walks in a graph. 

In [7]:
from AnonymousWalkKernel import AnonymousWalks

G = gk.graphs[0]
aw = AnonymousWalks(G)
length = 3 # length of anonymous walks

aw._all_paths(steps=length)
print('All possible anonymous walks of length {} (a.k.a embedding size)'.format(length))
print(aw.paths[length])

All possible anonymous walks of length 3
[[0, 1, 2, 0], [0, 1, 0, 1], [0, 1, 2, 1], [0, 1, 0, 2], [0, 1, 2, 3]]


We can get _exact_ distribution of anonymous walks in a graph for small lengths (e.g. $l <= 7$) using `embed` method.

In [23]:
import time

length = 4
start = time.time()
embedding, meta = aw.embed(steps = length, method = 'exact', keep_last=True, verbose=False)
finish = time.time()

aws = meta['meta-paths']
print('Computed Embedding of {} dimension in {:.3f} sec.'.format(len(aws), finish - start))
print()
print('Embedding:')
print(embedding)
print()
print('Anonymous Walks of length {}:'.format(length))
print(aws)

Computed Embedding of 15 dimension in 0.013 sec.

Embedding:
[0.08742619431025227, 0.09923510466988726, 0.09923510466988733, 0, 0, 0, 0, 0, 0.10782340311325822, 0.09923510466988726, 0.13560118089103612, 0, 0.0539452495974235, 0.12822061191626416, 0.18927804616210467]

Anonymous Walks of length 4:
[[0, 1, 0, 1, 0], [0, 1, 2, 1, 0], [0, 1, 0, 2, 0], [0, 1, 2, 3, 0], [0, 1, 2, 0, 1], [0, 1, 0, 2, 1], [0, 1, 2, 3, 1], [0, 1, 2, 0, 2], [0, 1, 0, 1, 2], [0, 1, 2, 1, 2], [0, 1, 2, 3, 2], [0, 1, 2, 0, 3], [0, 1, 2, 1, 3], [0, 1, 0, 2, 3], [0, 1, 2, 3, 4]]


We can also use _sampling_ approach to get distribution. There are two ways to control the number of samples:
1. $\varepsilon$ and $\delta$ to control $\varepsilon$-proximity to the true anonymous walk  distribution,
2. Monte-Carlo simulations MC. 

In [ ]:
length = 10
start = time.time()
embedding, meta = aw.embed(steps = length, method = 'sampling', keep_last=True, verbose=False, MC=100)
finish = time.time()


aws = meta['meta-paths']
print('Computed Embedding of {} dimension in {:.3f} sec.'.format(len(aws), finish - start))
print()
print('Embedding:')
print(embedding)



### Getting feature-based embeddings of all graphs

We can embed all graphs using `GraphKernel` class.

In [46]:
length = 3
gk.embed_graphs(steps = length, keep_last=True)

E = gk.embeddings
print('Embeddings matrix of shape: {} x {}'.format(E.shape[0], E.shape[1]))
print(E[:10, :])

Using exact method to get graph embeddings
Processing 0 graph
Processing 100 graph
Embeddings matrix of shape: 188 x 5
[[ 0.          0.1952496   0.25241546  0.22745572  0.32487923]
 [ 0.          0.22792023  0.3005698   0.1994302   0.27207977]
 [ 0.          0.19590643  0.26169591  0.2251462   0.31725146]
 [ 0.          0.1952496   0.25322061  0.22745572  0.32407407]
 [ 0.          0.21132898  0.26252723  0.22657952  0.29956427]
 [ 0.          0.20185185  0.26342593  0.22037037  0.31435185]
 [ 0.          0.2137037   0.28444444  0.21074074  0.29111111]
 [ 0.          0.20077973  0.25584795  0.22612086  0.31725146]
 [ 0.          0.19477513  0.26190476  0.21990741  0.3234127 ]
 [ 0.          0.20697168  0.27668845  0.21786492  0.29847495]]


### Getting kernel matrix from embeddings
Once you have a matrix of graph embeddings (or any feature matrix) you can compute kernel matrix from it. There are 3 types of kernels:
1. RBF (a.k.a. gaussian): $\exp (\dfrac{\Vert x - y\Vert^2}{2\sigma^2})$
2. Polynomial: $(xy + c)^d$
3. Dot: $xy$

In [47]:
gk.embeddings = E
gk.kernel_matrix(kernel_method='rbf', build_embeddings=False)
K = gk.K
print('Kernel matrix has shape: {} x {}'.format(K.shape[0], K.shape[1]))
print(K[:10, :])

Kernel matrix has shape: 188 x 188
[[ 1.          0.99306474  0.99984994 ...,  0.99652161  0.99694477
   0.99728956]
 [ 0.99306474  1.          0.99477584 ...,  0.99910552  0.99834351
   0.99863453]
 [ 0.99984994  0.99477584  1.         ...,  0.99756893  0.99783446
   0.99823587]
 ..., 
 [ 0.99989768  0.99452552  0.9999411  ...,  0.99760042  0.99795464
   0.99822765]
 [ 0.99985061  0.99436795  0.99993327 ...,  0.99712994  0.9972739
   0.9977629 ]
 [ 0.99848539  0.99795639  0.99924749 ...,  0.99938722  0.9993166
   0.99966677]]


### Evaluating performance
To evaluate performance of kernel matrix we need to read graph classes and use `Evaluation` class. It runs SVM algorithm and performs k-fold cross-validation on train-val-test splits. 

In [52]:
import numpy as np
# reading classes of graphs
y = []
with open('../bio/mutag/labels.txt') as f:
    for line in f:
        y.extend(list(map(int, line.strip().split())))
y = np.array(y)
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1]


In [57]:
from AnonymousWalkKernel import Evaluation
k = 10
ev = Evaluation(matrix = K, labels = y, verbose = False)
accuracies = ev.evaluate(k=k)
print('Performed {}-fold validation and scored {:.2f}% mean accuracy in graph classification'.format(k, 100*np.mean(accuracies)))

Performed 10-fold validation and scored 67.05% mean accuracy in graph classification


### Getting distributed Anonymous Walk Embeddings (AWE)

Distributed model has a variety of parameters. The inputs are: 
* dataset: name of the dataset and corresponding name of the folder.
* batch_size: number of batches per iteration of AWE model.
* window_size: number of context words.
* concat: Concatenate context words or not.
* embedding_size_w: embedding size of word
* embedding_size_d: embedding size of document
* loss_type: sampled softmax or nce
* num_samples: number of (negative) samples for every target word.
* optimize: SGD or Adagrad
* learning_rate: learning rate of the model
* root: root folder of the dataset
* ext: extension of files with graphs (e.g. graphml)
* steps: length of anonymous walk
* epochs: number of epochs for iterations
* batches_per_epoch: number of batches per epoch for each graph
* candidate_func: None (loguniform by default) or uniform
* graph_labels: None, edges, nodes, edges_nodes

In [61]:
from AnonymousWalkEmbeddings import AWE

dataset = 'mutag'

batch_size = 100
window_size = 8
embedding_size_w = 128
embedding_size_d = 128
num_samples = 32

concat = False
loss_type = 'sampled_softmax'
optimize = 'Adagrad'
learning_rate = 1.0
root = '../bio/'
ext = 'graphml'
steps = 7
epochs = 1
batches_per_epoch = 50
candidate_func = None
graph_labels = None

In [62]:
awe = AWE(dataset = dataset, batch_size = batch_size, window_size = window_size,
                  embedding_size_w = embedding_size_w, embedding_size_d = embedding_size_d,
                  num_samples = num_samples, concat = concat, loss_type = loss_type,
                  optimize = optimize, learning_rate = learning_rate, root = root,
                  ext = ext, steps = steps, epochs = epochs, batches_per_epoch = batches_per_epoch,
                  candidate_func = candidate_func, graph_labels=graph_labels)

Number of graphs: 188
Number of words: 877


In [63]:
print()
start2emb = time.time()
awe.train() # get embeddings
finish2emb = time.time()
print()
print('Time to compute embeddings: {:.2f} sec'.format(finish2emb - start2emb))


Initialized
Epoch: 0
Graph 0: 23 nodes
Time: 1.0299999713897705
Graph 1: 26 nodes
Average loss at step 100: 8616.452126
Graph 2: 19 nodes
Graph 3: 23 nodes
Average loss at step 200: 7335.297304
Graph 4: 17 nodes
Graph 5: 20 nodes
Average loss at step 300: 5466.474916
Graph 6: 25 nodes
Graph 7: 19 nodes
Average loss at step 400: 4691.216660
Graph 8: 28 nodes
Graph 9: 17 nodes
Average loss at step 500: 4535.514004
Graph 10: 15 nodes
Time: 0.9779996871948242
Graph 11: 12 nodes
Average loss at step 600: 4194.634135
Graph 12: 21 nodes
Graph 13: 23 nodes
Average loss at step 700: 3422.259841
Graph 14: 17 nodes
Graph 15: 25 nodes
Average loss at step 800: 3062.469240
Graph 16: 21 nodes
Graph 17: 19 nodes
Average loss at step 900: 3103.475781
Graph 18: 20 nodes
Graph 19: 21 nodes
Average loss at step 1000: 2833.302526
Graph 20: 15 nodes
Time: 1.5069999694824219
Graph 21: 20 nodes
Average loss at step 1100: 3270.603173
Graph 22: 22 nodes
Graph 23: 20 nodes
Average loss at step 1200: 2971.33030

In [68]:
E = awe.graph_embeddings
print('Shape of embedding matrix: {}'.format(E.shape))
gk.embeddings = E
gk.kernel_matrix(build_embeddings=False, kernel_method='poly', d = 3, c = 1)
K = gk.K
print('Shape of kernel matrix: {}'.format(K.shape))
print()
print(K[:10, :])

Shape of embedding matrix: (188, 128)
Shape of kernel matrix: (188, 188)

[[ 8.00000143  1.69091181  1.22703558 ...,  0.73225611  1.25856365
   1.14092373]
 [ 1.69091181  8.          1.82415656 ...,  1.31729895  1.75605998
   1.01578914]
 [ 1.22703558  1.82415656  8.00000143 ...,  1.21977914  1.33776616
   1.36803289]
 ..., 
 [ 0.87311691  0.89288239  1.32151768 ...,  0.96594554  1.2753261
   1.25214279]
 [ 1.49640147  1.91465694  1.78598061 ...,  1.48569894  1.57869433
   1.51887444]
 [ 1.27794699  0.81278121  1.42610971 ...,  0.93055158  0.80055678
   1.20529534]]


### Getting kernel matrix from embeddings
If you have an embedding matrix `E` obtained by any model, you can compute a kernel matrix using `GraphKernel` class. The choice for kernel function is among `rbf`, `poly`, and `dot`. 

To provide access to embeddings matrix, you have 2 choices. 

In [12]:
from AnonymousWalkKernel import GraphKernel

gk = GraphKernel()

# Option 1. Load embeddings from graph kernel method.  
E = gk.load_embeddings('mutag_embeddings.txt.npz')
print('Option 1')
print(E[:2, :2])

# Option 2. Assign the matrix. 
random_matrix = np.random.random((10,20))
gk.embeddings = random_matrix
E = gk.embeddings
print('Option 2')
print(E[:2, :2])

Option 1
[[ 0.42270531  0.57729469]
 [ 0.42735043  0.57264957]]
Option 2
[[ 0.50164817  0.70202204]
 [ 0.62296715  0.76445566]]


In [16]:
# RBF kernel with sigma = 0.5
# build_embeddings = False reuses embeddings already provided to the class
gk.kernel_matrix(kernel_method = 'rbf', build_embeddings=False, sigma=0.5)
K = gk.K
print('RBF:')
print(K)

RBF:
[[  1.00000000e+00   3.23452037e-04   7.63190047e-04   7.93150032e-04
    1.41839034e-03   8.36838298e-03   1.13285565e-03   1.19557048e-03
    4.01878268e-03   1.24549318e-02]
 [  3.23452037e-04   1.00000000e+00   4.14719223e-02   7.17735258e-03
    1.43788276e-03   7.16673760e-04   2.37373897e-03   7.04821845e-05
    1.10784993e-02   9.07881791e-04]
 [  7.63190047e-04   4.14719223e-02   1.00000000e+00   1.13103205e-02
    9.94277249e-03   3.00793166e-04   5.37412635e-03   7.81132994e-05
    9.15431570e-04   3.78065255e-03]
 [  7.93150032e-04   7.17735258e-03   1.13103205e-02   1.00000000e+00
    6.60094664e-03   1.24185540e-03   7.13997877e-03   1.45584113e-03
    5.01503052e-03   5.65735358e-04]
 [  1.41839034e-03   1.43788276e-03   9.94277249e-03   6.60094664e-03
    1.00000000e+00   8.10755000e-04   1.88409379e-02   1.15536122e-03
    2.34643623e-03   4.07424891e-03]
 [  8.36838298e-03   7.16673760e-04   3.00793166e-04   1.24185540e-03
    8.10755000e-04   1.00000000e+00   8.

In [17]:
# Poly kernel with c = 1 and d = 3
gk.kernel_matrix(kernel_method = 'poly', build_embeddings=False, c = 1, d = 3)
K = gk.K
print('Poly:')
print(K)

Poly:
[[ 448.13874358  169.63642671  137.71748084  192.37284357  121.56874438
   243.89311992  262.63471908  319.94021313  271.95135398  303.0956935 ]
 [ 169.63642671  411.35401882  222.02137041  240.14370967  114.05193019
   168.72690073  272.25262006  218.60562066  290.72940968  211.16510102]
 [ 137.71748084  222.02137041  245.87062438  189.43584003  106.952873
   105.32427953  226.90412862  163.08723345  158.78946611  187.67276914]
 [ 192.37284357  240.14370967  189.43584003  415.61100538  144.15292043
   182.86489226  310.19567877  313.57382669  267.00197238  200.14318122]
 [ 121.56874438  114.05193019  106.952873    144.15292043  169.50270936
    97.66603305  221.19069147  192.27923019  147.63205919  155.61563733]
 [ 243.89311992  168.72690073  105.32427953  182.86489226   97.66603305
   378.70313589  170.25953461  326.55957421  239.27035335  289.46608771]
 [ 262.63471908  272.25262006  226.90412862  310.19567877  221.19069147
   170.25953461  624.36304928  448.04640126  349.02543

### Repro